# DepthAI - Start Here

This notebook will read and display frames from the rgb camera.

In [ ]:
# these modules are for DepthAI
import blobconverter
import cv2
import depthai
import numpy as np

In [ ]:
# these matplotlib modules will enable us to display the image in the Jupyter notebook
import matplotlib.pyplot as plt
import matplotlib.image as img

In [ ]:
# these modules will enable us to build a simple event driven UI in the Jupyter notebook
import ipywidgets as widgets
import threading
from IPython.display import display, Image

In [ ]:
# Create a pipeline that tells DepthAI what operations to perform while running.
pipeline = depthai.Pipeline()
assert pipeline is not None

In [ ]:
# Add the color camera node to the pipeline.
colorCamera = pipeline.createColorCamera()
assert colorCamera is not None

In [ ]:
# Set the preview size to 300x300.  
# This frame will be available as the 'preview' output of the node.
colorCamera.setPreviewSize(300, 300)  
colorCamera.setInterleaved(False)

In [ ]:
# XLinkOut is a "way out" from the device. Any data you want to transfer to host need to be send via XLink.
xoutRgb = pipeline.createXLinkOut()
assert xoutRgb is not None

# For the rgb camera output, we want the XLink stream to be named "rgb"
xoutRgb.setStreamName("rgb")

# Linking camera preview to XLink input, so that the frames will be sent to host
colorCamera.preview.link(xoutRgb.input)

### The Pipeline is now fully constructed.

In [ ]:
# Create a function to be called in the thread that will be created in the next notebook cell.
def showVideo(button):
    
    # ask the device to run the pipeline.
    with depthai.Device(pipeline) as device:

        # From this point, the Device will be in "running" mode and will start sending data via XLink.
        queueRgb = device.getOutputQueue("rgb")

        # Frame will be an image from "rgb" stream.
        frame = None    

        # Main host-side application loop
        while True:
            
            # we try to fetch the data from nn/rgb queues. 
            # tryGet() will return either the data packet or None if there isn't any.
            inRgb = queueRgb.tryGet()

            if inRgb is not None:
                # If the packet from RGB camera is present, 
                # we're retrieving the frame in OpenCV format using getCvFrame().
                frame = inRgb.getCvFrame()

            if frame is not None:
                
                # adjust the frame size for conversion and display in the notebook.
                rows, columns, _ = frame.shape
                resizedFrame = cv2.resize(frame, (int(columns/2), int(rows/2)))
                resizedFrame = cv2.flip(resizedFrame, 1)
                
                # convert the frame from a OpenCV numpy array to a jpeg 
                # so that we can display it in the Jupyter notebook.
                _, jpg = cv2.imencode('.jpeg', resizedFrame)
                
                # show the frame on the screen.
                display_handle.update(Image(data=jpg.tobytes()))
                if stopButton.value==True:
                    break

In [ ]:
# define the button widget
stopButton = widgets.ToggleButton(
    value=False,
    description='Stop',
    disabled=False,
    button_style='danger',
    tooltip='Stop the Video',
    icon='square'
)

# initialize the display window
display_handle=display(None, display_id=True)
# display the stop button
display(stopButton)
# create a thread that calls the showVideo() function above
thread = threading.Thread(target=showVideo, args=(stopButton,))
thread.start()